In [ ]:
# google search part
for faculty in json_data:
  name = faculty['name']
  institution_name = faculty['affiliation']['name']
  #print(name)
  # step1  : perform a google search
  google_search_url = f"https://www.google.com/search?q=" + name
  response = requests.get(google_search_url)
  if response.status_code ==200:

    # step 2 : parse the google search results page
    soup = BeautifulSoup(response.content, 'html.parser')
    #print(soup.prettify())
    # extract url of the first search result ( might adjust)
    search_results = soup.find_all('a',href=True)

    scholarly_url = None
    #print(search_results)
    for result in search_results:
      url_with_params=result['href']
      parsed_url = urllib.parse.parse_qs(urllib.parse.urlparse(url_with_params).query)
      url = parsed_url.get('q',[''])
     # print(url)
      if 'https://scholar.google.com/citations' in url[0]:
        scholarly_url=url[0]
        #print(scholarly_url)

        break

    if scholarly_url:
      time.sleep(5)
      scholarly_response = requests.get(scholarly_url,headers=headers)
      if scholarly_response.status_code==200:
        scholarly_soup = BeautifulSoup(scholarly_response.content, 'html.parser')
        # extract author's name
        author_name = scholarly_soup.find('div',id='gsc_prf_in').text.strip()

         # Affiliation (if available)
        affiliation_elements = scholarly_soup.find_all('div',class_='gsc_prf_il')
        if affiliation_elements:
          author_affiliation = affiliation_elements[0].text.strip()
          universities = [univ.strip() for univ in author_affiliation.split(',')]

          for university in universities:
            if "University" in university:
              university_name = university
              break
            else:
              university_name = universities[0]

          email_text = affiliation_elements[1].get_text()
          if email_text:
            domain_match = re.search(r'Verified email at (\S+)',email_text)
            if domain_match :
              domain_part = domain_match.group(1)
            else:
              domain_part = None

          if domain_part and university_name and domain_part in institution_name:
            score = 100
            print('domain is in institution')
          else:
            score = fuzz.token_set_ratio(institution_name,university_name)
          if score<70:
            with open('check_affiliation.txt', 'a') as f:
                  json.dump(name, f)
                  json.dump(institution_name, f)
                  json.dump(author_name, f)
                  json.dump(university_name, f)
                  json.dump(domain_part,f)
                  f.write('\n')
            pass


        #Author's image URL (if available)
        image_element = scholarly_soup.find('img',id='gsc_prf_pup-img')
        if image_element:
          author_image_url = image_element['src']

        # Publication names
       # publication_titles=[]
       # publications = scholarly_soup.find_all('tr',class_='gsc_a_tr')
       # for publication in publications:
       #   title_element = publication.find('a',class_='gsc_a_at')
       #   title = title_element.text.strip() if title_element else None
          #print(title)
      #    if title:
      #      title = title.split('\n',1)[0].strip()
      #    publication_titles.append(title)
          ''' author,journal, year
            authors_element = row.find('div', class_='gs_gray')
            authors = authors_element.text.strip() if authors_element else None

            journal_element = authors_element.find_next_sibling('div', class_='gs_gray')
            journal = journal_element.text.strip() if journal_element else None

            year_element = row.find('span', class_='gsc_a_h')
            year = year_element.text.strip() if year_element else None

            '''

        filename = "temp_image"
        with open('scholar.txt','a') as f:
          json.dump(author_name,f)
          if affiliation_elements:####should change
            json.dump(university_name,f)###should change
          if image_element:
            # json.dump(author_image_url,f)
            response = requests.get(author_image_url)

            if response.status_code == 200:
              with open(filename, "wb") as f1:
                f1.write(response.content)
                # print("Image downloaded successfully as", filename)
                obj = DeepFace.analyze(filename, actions=["age", "gender", "race"])
                gender = obj[0]['dominant_gender']
                race = obj[0]['dominant_race']
                # print(gender)
                json.dump(gender, f)
                json.dump(race, f)
            else:
              print("Failed to download the image.")
          f.write('\n')

    else:
      # handle the case when the scholarly website link is not found
      with open('scholar_not_found.txt','a') as f:
        json.dump(name,f)
        json.dump(institution_name,f)
        f.write('\n')
  else:
    # google search fails
    with open('google_not_found.txt','a') as f:
      json.dump(name,f)
      json.dump(institution_name,f)
      f.write('\n')

In [ ]:
# google second search part: name + institution_name

for faculty in corrected_data:
  name = faculty['name']
  institution_name = faculty['institution']
  print(name)
  # step1  : perform a google search
  google_search_url = f"https://www.google.com/search?q=" + name+institution_name
  response = requests.get(google_search_url)
  if response.status_code ==200:

    # step 2 : parse the google search results page
    soup = BeautifulSoup(response.content, 'html.parser')
    #print(soup.prettify())
    # extract url of the first search result ( might adjust)
    search_results = soup.find_all('a',href=True)

    scholarly_url = None
    #print(search_results)
    for result in search_results:
      url_with_params=result['href']
      parsed_url = urllib.parse.parse_qs(urllib.parse.urlparse(url_with_params).query)
      url = parsed_url.get('q',[''])
     # print(url)
      if 'https://scholar.google.com/citations' in url[0]:
        scholarly_url=url[0]
        #print(scholarly_url)

        break

    if scholarly_url:
      time.sleep(5)
      scholarly_response = requests.get(scholarly_url,headers=headers)
      if scholarly_response.status_code==200:
        scholarly_soup = BeautifulSoup(scholarly_response.content, 'html.parser')
        # extract author's name
        author_name = scholarly_soup.find('div',id='gsc_prf_in').text.strip()

         # Affiliation (if available)
        affiliation_elements = scholarly_soup.find_all('div',class_='gsc_prf_il')
        if affiliation_elements:
          author_affiliation = affiliation_elements[0].text.strip()
          universities = [univ.strip() for univ in author_affiliation.split(',')]

          for university in universities:
            if "University" in university:
              university_name = university
              break
            else:
              university_name = universities[0]

          email_text = affiliation_elements[1].get_text()
          if email_text:
            domain_match = re.search(r'Verified email at (\S+)',email_text)
            if domain_match :
              domain_part = domain_match.group(1)
            else:
              domain_part = None

          if domain_part and university_name and domain_part in university_name:
            score = 100
          else:
            score = fuzz.token_set_ratio(institution_name,university_name)
          if score<70:
            with open('check_affiliation.txt', 'a') as f:
                  json.dump(name, f)
                  json.dump(institution_name, f)
                  json.dump(author_name, f)
                  json.dump(university_name, f)
                  json.dump(domain_part,f)
                  f.write('\n')
            pass


        #Author's image URL (if available)
        image_element = scholarly_soup.find('img',id='gsc_prf_pup-img')
        if image_element:
          author_image_url = image_element['src']

        # Publication names
        #publication_titles=[]
        #publications = scholarly_soup.find_all('tr',class_='gsc_a_tr')
        #for publication in publications:
        #  title_element = publication.find('a',class_='gsc_a_at')
        #  title = title_element.text.strip() if title_element else None
          #print(title)
       #   if title:
       #     title = title.split('\n',1)[0].strip()
       #   publication_titles.append(title)
          ''' author,journal, year
            authors_element = row.find('div', class_='gs_gray')
            authors = authors_element.text.strip() if authors_element else None

            journal_element = authors_element.find_next_sibling('div', class_='gs_gray')
            journal = journal_element.text.strip() if journal_element else None

            year_element = row.find('span', class_='gsc_a_h')
            year = year_element.text.strip() if year_element else None

            '''

        with open('scholar.txt','a') as f:
          json.dump(author_name,f)
          if affiliation_elements:####should change
            json.dump(university_name,f)###should change
          if image_element:
            # json.dump(author_image_url,f)
            response = requests.get(author_image_url)

            if response.status_code == 200:
              with open(filename, "wb") as f1:
                f1.write(response.content)
                # print("Image downloaded successfully as", filename)
                obj = DeepFace.analyze(filename, actions=["age", "gender", "race"])
                gender = obj[0]['dominant_gender']
                race = obj[0]['dominant_race']
                # print(gender)
                json.dump(gender, f)
                json.dump(race, f)
            else:
              print("Failed to download the image.")
          f.write('\n')

    else:
      # handle the case when the scholarly website link is not found
      with open('scholar_not_found_second_query.txt','a') as f:
        json.dump(name,f)
        json.dump(institution_name,f)
        f.write('\n')
  else:
    # google search fails
    with open('google_search_error.txt','a') as f:
      json.dump(name,f)
      json.dump(institution_name,f)
      f.write('\n')


In [ ]:
# for people who haven't image on google and those who ia found by DBLP and OpenAlex
corrected_data_2 = []
with open('scholar_not_found_second_query.txt', 'r') as file:
    for line in file:
        line = line.strip()
        elements = line.split('""')  # Split the line using the specified delimiter
        if len(elements) ==2 :  # Check if there are 5 elements
            cleaned_elements = [element.strip(' ').strip('"') for element in elements]
            name = cleaned_elements[0]
            # use name-based method to identify gender and name, see name_based_identification.py
            corrected_data_2.append({
                "name": cleaned_elements[0],
                "institution": cleaned_elements[1]
            })